# Language Models

This notebook explores using character-level n-grams to model language. We will learn how to train models by analyzing a body of text, and then use them for a fun task: generating new language in the style of a model.

A brief review of Python strings: recall that strings are sequence-objects. This means that they behave like tuples: they support indexing/slicing and can be iterated over (which occurs character-by-character). Naturally, the length of a string reflects the number of characters in a string. Special characters, like the line-break `\n`, is a single character.

Python provides excellent, efficient string-methods. Initialize some string, `x = "moo"`. And then use tab-completing (`x.<tab>`) to view the list of built-in string methods. Be sure to make use of these whenever appropriate.

## 0) Imports

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
def unzip(pairs):
    """Splits list of pairs (tuples) into separate lists.
    
    Example: pairs = [("a", 1), ("b", 2)] --> ["a", "b"] and [1, 2]
    
    This should look familiar from our review back at the beginning of week 1
    :)
    """
    return tuple(zip(*pairs))

## 1 Most frequent letters (in English)

Ever wonder why the bonus round of Wheel of Fortune automatically gives the contestant the letters R, S, T, L, N, and E (before letting them choose an additional 3 consonants and 1 vowel)? Or wonder why the letters J, X, Q, and Z are worth so much in Scrabble?

Let's find out by analyzing a particular corpus of English text: Wikipedia. Thanks to Evan Jones for providing a clean text-only version of top Wikipedia articles (based on the Wikipedia "release version" project): http://www.evanjones.ca/software/wikipedia2text.html

Load the entire contents of "wikipedia2text-extracted.txt" into a single string. Because some of these articles contain non-[ASCII](http://www.asciitable.com/) characters (for instance, some Chinese characters), you will need to open the file in binary-read mode: `mode='rb'`. Instead of reading in a typical string, this will read in a `bytes` object, which is simply your machine's memory-encoding for the characters. To make a long story short, you can simply call the method `decode` on this bytes-instance to decode the bytes into a familiar string. E.g. 

```python
# having read wikipedia2text-extracted.txt in binary-read mode
# into the variable `wiki_bytes`
wiki_str = wiki_bytes.decode()
```
After decoding from bytes to a string, make all of the characters in the string lowercase (you do not need a for-loop for this!)

Confirm that there are over 63 million characters total, in the string. 

In [ ]:
# SOLUTION
path_to_wikipedia = "wikipedia2text-extracted.txt"

# YOUR CODE HERE

print(str(len(wikipedia)) + " character(s)")

Print out the first 500 characters of text.

In [ ]:
# SOLUTION

### 1.1 Count letters in text

We want to count the occurence of every letter-character in our corpus. First, count up the occurence of every character (including punctuation and special characters, like `\n`)

Hint: Python has a `Counter` object in its `collections` module (which is a built-in module in Python, similar to `itertools`). You should be able to produce the count for every character in the file in one line. This should take roughly 5 seconds. Using a for-loop will take roughly 10x longer!


In [ ]:
# SOLUTION

Now we want a list of character-count tuples sorted in descending order of count. However, we want to filter out all of the non-letter characters. Thus our list should have a length of 26 (since we cast all of the letters to be lower-case).

If you made use of the `Counter` class, then there is a nice instance-method that you can make use of that takes care of sorting the character-count tuples so that the most-common occurences come first. You will need to filter out the non-letter characters.

In [ ]:
# SOLUTION

Create a variable called `freqs`, which is also a list of tuples, but instead containing character-count pairs, it contains character-frequency pairs. Frequency is the ratio of the letter-count to the total number of letters in the corpus. It should end up looking something like:

    [('e', 0.12081350306248849),
     ('t', 0.088397321263964282),
     ('a', 0.0865778000521603),
     ...
     ('q', 0.0010429083984244488)]
     
You should **not** iterate over the entire corpus to get the total letter count.

In [ ]:
# SOLUTION

Confirm that the frequencies total to 1 (within numerical precision).

In [ ]:
# SOLUTION

### 1.2 Plot letter frequency histogram

Using 
```
fig, ax = plt.subplots()
```
Look up the documentation for `ax.bar` (use shift-tab in your Jupyter notebook!) to plot a histogram of the characters and their frequencies. The x-axis should order the characters in decreasing frequency.

Use `ax.set_xticks` to specify the sequence of x-ticks to use (i.e. [0, 26) ), and use
`ax.set_xticklabels` to provide the custom labels for the ticks along the x-axis (i.e. 'e', 't', 'a' ...)

Be sure to provide a title and a descriptive label for the y-axis.

In [ ]:
# SOLUTION

## 2 Most frequent words (in English)

Let's move up a level from characters and look at the distribution the English words. Returning to the full wikipedia corpus, which we have stored as a lower-cased string. Let's tokenize the corpus.

### 2.1 (Simple) Tokenization

For now we'll just apply a simple tokenization scheme: splitting on whitespace (spaces, tabs, newlines, etc.).

Without using a for-loop, produce a list of the tokens from the corpurs, and print out the first 10 tokens along with the total number of tokens.

In [ ]:
# SOLUTION

As you did above with characters, count the occurences of all of the different words. 

In [ ]:
# SOLUTION

Finally, display the top 20 most-common words.

In [ ]:
# SOLUTION

What do you notice about this list? Do you see any people or places in this list? Can you discern anything about the content of the articles from these words (other than the fact that they predominantly contain English)? These words, referred to as "stop words" in NLP applications, are ubiquitous to modern English, thus we will many times want to filter them out, so that we can get at the "meaningful" words in a corpus.

See https://en.wikipedia.org/wiki/Stop_words for more details. 

Tokenizing our corpus and identifying the so-called stop words will be critical preprocessing steps for us to train our first language model!

## 3 Training an n-gram language model

Inspired by Yoav Goldberg blog, which in turn was inspired by a [post from Andrej Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) that you saw in the Perspective on Machine Learning module, we will train our own n-gram language model.

Character-based n-gram language models aim to guess the next letter based on seeing the previous (n-1) letters. (The assumption that the probability of seeing a litter only depends on a certain finite number of previous letters is an example of a Markov assumption. See https://en.wikipedia.org/wiki/Markov_property for more information.)

Just as we did above, we will want to take a counter-instance of letter-count pairs, and convert them to letter-frequency pairs. Using your code from earlier as reference, provide the body for the following function. Do **not** filter any characters this time.

In [ ]:
# SOLUTION

def normalize(counter):
    """ Convert counter to a list of (letter, frequency) pairs, sorted in descending order of frequency.
    
        Parameters
        -----------
        counter: A Counter-instance

        Returns
        -------
        A list of tuples - (letter, frequency) pairs. 
        
        For example, if counter had the counts:
        
            {'a': 1, 'b': 3}
        
        `normalize(counter)` will return:
        
            [('b', 0.75), ('a', 0.25)]
    """
    pass

In the following, we will want to make some serious use of Python's `collections` module. Not only do we want to use the `Counter` class again, we also will want to use a `defaultdict`. 

Unlike with a typical dictionary, a default-dictionary is told to generatre a default value whenever it encounters a new key. For instance, suppose you wanted a dictionary that mapped keys to lists, which we populate with some items. With a standard dictionary, we'd need the following logic:

```python
if key not in my_dict:
    my_dict[key] = []  # create an empty list for a new key
mydict[key].append(item)
```

Instead, we can use `defaultdict(list)`, which, upon encountering a new key, will automatically create a default value (using `list()`). Thus the preceeding code becomes:

```python
my_defaultdict[key].append(item)
```

Whatever object you pass to `defaultdict` to initialize it, is what is used to create the default value for that dictionary. Otherwise, `defaultdict` behaves precisely like a dictionary.

Now we'll create the function to actually analyze the n-grams that occur in a text. We'll slide a history of length (n - 1) along and increase the count of the character that follows it. After counting is done, we'll normalize the counts for each history to convert to frequencies, which we can interpret as probabilities.

To deal with the initial character(s) not having enough history, we'll pad the text with a dummy character, "~".

Here's an illustration of the process for the text "cacao":

    history is "~~", next char is "c", increment counts["~~"]["c"]
    history is "~c", next char is "a", increment counts["~c"]["a"]
    history is "ca", next char is "c", increment counts["ca"]["c"]
    history is "ac", next char is "a", increment counts["ac"]["a"]
    history is "ca", next char is "o", increment counts["ca"]["o"]

So we will want to create a default dictionary, whose default value is an empty `Counter` instance. I.e. `model = defaultdict(Counter)`. Thus `model[history]` will return the counter for that history. You can then update that counter with the character that proceedes the history: `history_counter[char] += 1`

In [ ]:
def train_lm(text, n):
    """ Train character-based n-gram language model.
        
        This will learn: given a sequence of n-1 characters, what the probability
        distribution is for the n-th character in the sequence.
        
        For example if we train on the text:
            text = "cacao"
        
        Using a n-gram size of n=3, then the following dict would be returned:
        
            {'ac': [('a', 1.0)],
             'ca': [('c', 0.5), ('o', 0.5)],
             '~c': [('a', 1.0)],
             '~~': [('c', 1.0)]}

        Tildas ("~") are used for padding the history when necessary, so that it's 
        possible to estimate the probability of a seeing a character when there 
        aren't (n - 1) previous characters of history available.
        
        So, according to this text we trained on, if you see the sequence 'ac',
        our model predicts that the next character should be 'a' 100% of the time.
        
       For generatiing the padding, recall that Python allows you to generate 
        repeated sequences easily: 
           `"p" * 4` returns `"pppp"`
           
        Parameters
        -----------
        text: str 
            A string (doesn't need to be lowercased).
        n: int
            The length of n-gram to analyze.
        
        Returns
        -------
        A dict that maps histories (strings of length (n-1)) to lists of (char, prob) 
        pairs, where prob is the probability (i.e frequency) of char appearing after 
        that specific history. For example, if

    """
    # initialize the model
    # create the initial history
    
    # Get the next character in the text
    # Given the history, update the counter
    # Revise the history
    # Repeat
    
    # Use your `normalize` function to convert the history -> char-count dictionary
    # to a history -> char-frequency dictionary. Return this dictionary
    pass

Test train_lm() on "cacao", using n=3. Should get same result as in comment.

In [ ]:
# SOLUTION

Now let's test our function on more serious example: a small snippet of text from "The Cat in the Hat" by Dr. Seuss. Train on the following text with `n=3`.

In [ ]:
text = """The sun did not shine, it was too wet to play, 
so we sat in the house all that cold, cold wet day. 
I sat there with Sally. We sat here we two 
and we said 'How we wish we had something to do.'"""

In [ ]:
# SOLUTION

Print out the model's predictive probability distribution, give the history `"~~"`.

The following should show a probability of 1.0 for the letter "T", since that's the only starting letter that the model has ever seen (i.e., with no history, indicated by `"~~"`).

In [ ]:
# SOLUTION

Print the distribution for the history `'~T'`

The following should show a probability of 1.0 for "h", since that's the only letter the model has seen after a history of "~T".

In [ ]:
# SOLUTION

Print the distribution for the history `'th'`

This last example should give a probability distribution of the characters "e", "a", " ", and "i", since those four characters all were observed to follow "th" in the text (with "e" occurring most often).

In [ ]:
# SOLUTION

## 4 Generating text

A fun thing to do with language models is to generate random text in the style of the model by generating letters using the learned probability distributions. 

First we'll create a function to generate a single letter given a particular history.

Hint: `np.random.choice(choices, p=probabilities)` will return an element from choices according to the specified probabilities. For example, `np.random.choice(["a", "b"], [0.25, 0.75])` will return an "a" 25% of the time and a "b" 75% of the time.

In [ ]:
def generate_letter(lm, history):
    """ Randomly picks letter according to probability distribution associated with 
        the specified history.
    
        Note: returns dummy character "~" if history not found in model.
    
        Parameters
        ----------
        lm: Dict[str, Tuple[str, float]] 
            The n-gram language model. I.e. the dictionary: history -> (char, freq)
        
        history: str
            A string of length (n-1) to use as context/history for generating 
            the next character.
        
        Returns
        -------
        str
            The predicted character. '~' if history is not in language model.
    """
    pass

Use the model that was trained on Dr. Seuss to generate a character, given the history `"th"`.

The following should generate "e", "a", " ", or "i", since those are the only four characters that followed "th".

In [ ]:
# SOLUTION

Repeat this, by generate 10 distinct samples

The following generates several possible next characters to get a sense of the distribution. "e" should appear more frequently than the other characters on average since it has a higher probability of following "th".

In [ ]:
# SOLUTION

Finally, we'll generate whole sequences of text according to the language model. The approach will be to start with no history ((n - 1) "~"s), generate a random letter, update the history, and repeat. In our example,

    history "~~" will generate 'T' (since only one possibility)
    history "~T" will generate 'h' (since only one possibility)
    history "Th" will generate 'e' (since only one possibility)
    history "he" could generate 'r' (out of multiple possibilities)
    history "er" will generate 'e' (since only one possibility)

and so on. The text generated so far would be "There".

In [ ]:
def generate_text(lm, n, nletters=100):
    """ Randomly generates nletters of text with n-gram language model lm.
    
        Parameters
        ----------
        lm: Dict[str, Tuple[str, float]] 
            The n-gram language model. I.e. the dictionary: history -> (char, freq)
        n: int
            Order of n-gram model.
        nletters: int
            Number of letters to randomly generate.
        
        Returns
        -------
        str
            Model-generated text.
    """
    # Create the initial history
    
    # Given the history
    # Generate the next character and "save" it
    # Update the history
    
    # Combine the generated characters into a single string. 
    # Experiment with "".join in a separate cell.  
    pass   

The following will generate 40 characters according to the 3-gram language model trained on the beginning of "The Cat in the Hat". It won't be very pretty... partly because of the short history length and also the small amount of training data.

In [ ]:
print(generate_text(lm3, 3, 40))

## 5 Generating "Shakespeare"

Lastly, we'll have some fun trying to generate text in the style of Shakespeare.

The next cell loads in Andrej Karpathy's shakespeare_input.txt file.

In [ ]:
path_to_shakespeare = "shakespeare_input.txt"
with open(path_to_shakespeare, "r") as f:
    shakespeare = f.read()
    
print(str(len(shakespeare)) + " character(s)")

chars = set(shakespeare)

"~" in chars  # checks if padding characters occur naturally in corpus

Now experiment with training models for various values of n (e.g., 3, 5, and 11) and generate some text (maybe 500 characters or so). You should find the 3-gram model to be a very bad speller; the 5-gram model a better speller, but not making much sense; and the 11-gram model looking pretty good (which is amazing considering the simplicity of the model).

Train your model on shakespear, with `n=3`.

In [ ]:
t0 = time.time()

# YOUR CODE HERE

t1 = time.time()
print("elapsed = " + str(t1 - t0) + "s")

Print 500 characters of text generated from this model.

In [ ]:
# SOLUTION

Repeat both of these steps with `n=5`.

In [ ]:
t0 = time.time()
# SOLUTION
t1 = time.time()
print("elapsed = " + str(t1 - t0) + "s")

In [ ]:
# SOLUTION

Repeat both steps for `n=11`.

In [ ]:
t0 = time.time()

# SOLUTION

t1 = time.time()
print("elapsed = " + str(t1 - t0) + "s")

In [ ]:
# SOLUTION

Idea for bonus fun: find some other text on the Internet to train models on, e.g., song lyrics, books from a particular author, etc.